<a href="https://colab.research.google.com/github/yingzibu/MOL2ADMET/blob/main/examples/Kp/Kp_data_20240122.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive

In [5]:
cd /content/drive/MyDrive/A_JAK_design/sun_lab_compound_pk/

/content/drive/MyDrive/A_JAK_design/sun_lab_compound_pk


In [6]:
ls

cal.py  compound_data.xlsx  drug_smile.xlsx  gfg.html  __pycache__/


In [7]:
!pip install rdkit --quiet
!pip install git+https://github.com/samoturk/mol2vec --quiet
from mol2vec.helpers import depict_identifier, plot_2D_vectors, IdentifierTable, mol_to_svg
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
def warn(*args, **kwargs):
    pass
warnings.warn = warn
from cal import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 25.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.1 MB/s eta 0:00:00


In [12]:
# compound_path = 'compound_data_01222024.xlsx'
# compounds = pd.read_excel(compound_path)
# compounds.fillna(method='ffill', inplace=True)
# names = [*set(compounds['Compound name '])]
# cols = compounds.columns

In [ ]:
compounds

In [15]:
smi_path = 'drug_smiles_01222024.xlsx'
smiles_df = pd.read_excel(smi_path)
length = len([*set(smiles_df['NAME'])])

try:
    assert length == smiles_df.shape[0]
except:
    print(f'there are multiple lines for drugs in file {smi_path}')
    for s in set(smiles_df['NAME']):
        if smiles_df.loc[smiles_df['NAME'] == s].shape[0] != 1:
            print(f'multi lines:  {s}      check',
                  smiles_df.loc[smiles_df['NAME'] == s].index.tolist())

compound_excel = [*set(compounds['Compound name '].tolist())]
print(len(compound_excel))

1382


In [16]:
compound_csv = [*set(smiles_df['NAME'])]
checks = []
checks_1_round = []
name_dict = {}

handled_num = 0
for i in compound_excel:
    if i not in compound_csv:
        checks.append(i)
        if '(' in i:
            # print(i)
            # print(i.split(' (')[0])
            if i.split(' (')[0] in compound_csv:
                name_dict[i] = i.split(' (')[0]
                handled_num += 1
        else: checks_1_round.append(i)

print(f'{len(checks)} compounds in excel but not in csv')
print(f'{handled_num} compounds in excel have (), and successfully matches csv')
print(f'{len(checks_1_round)} compounds are still need to be checked')
# print(checks)
print('proceed to check -1, -2 named drugs')
name_dict

491 compounds in excel but not in csv
5 compounds in excel have (), and successfully matches csv
486 compounds are still need to be checked
proceed to check -1, -2 named drugs


{'ARD-3474-WX (36)': 'ARD-3474-WX',
 'ARD-3474-WX (39)': 'ARD-3474-WX',
 'ARD-3474-WX (40)': 'ARD-3474-WX',
 'ARD-3474-WX (38)': 'ARD-3474-WX',
 'ARD-3474-WX (37)': 'ARD-3474-WX'}

In [17]:
'ARD-3474-WX (36)' in compound_csv

False

In [18]:
checks_2_round = []
name_end_char = []
check_convert_digit = []
handled_num = 0
checks_3_round = []
for i in checks_1_round:
    if i[-1].isdigit(): # check the name is end with a number
        # print(i)
        name_split = i.split('-')
        try:
            last_num_in_i = int(name_split[-1])
            # if the last num is larger than 20, check 2nd round
            if last_num_in_i > 20:
                checks_2_round.append(i)

            else:
                new_name = name_split[:-1]
                new_compound_name = '-'.join(new_name)

                if new_compound_name in compound_csv:
                    name_dict[i] = new_compound_name
                    handled_num += 1
                else:
                    # print(new_compound_name, i)
                    checks_3_round.append(i)
        except:
            check_convert_digit.append(i)
    else:
        name_end_char.append(i)

print(f'{len(checks_1_round)} compounds are being checked')
print(f'{handled_num} compounds names like -num could be found in csv file')
print(f'name dict is being updated')
# print(name_dict)

print(80 * '*')
print('CANNOT HANDLE FOLLOWING COMPOUNDS')

print(f'{len(checks_3_round)} compounds names like -num still cannot be found in csv file')
for j in checks_3_round:
    print(j)


print(f'{len(name_end_char)} compounds ends with character info cannot be found in csv')

for j in name_end_char:
    print(j)
print(f'{len(check_convert_digit)} compounds ends with num yet not in -num form')
print(check_convert_digit)

print(f'{len(checks_2_round)} compounds ends with num > 20')
for j in checks_2_round:
    print(j)

print(80 * '*')


486 compounds are being checked
445 compounds names like -num could be found in csv file
name dict is being updated
********************************************************************************
CANNOT HANDLE FOLLOWING COMPOUNDS
15 compounds names like -num still cannot be found in csv file
ERD-1297-RKA-1
ARD-3504-WX-1
ERD-12310-B-1
BD-7216-HJG-1
ERD-1233-RKA-11
ERD-1237-RKA-9
ERD-12310-B-2
EEDI-1251-1
ERD-1233-RKA-12
BD-7216-HJG-2
ERD-1237-RKA-8
BD-7216-HJG-3
ERD-1297-RKA-2
ERD-1233-RKA-10
ARD-3526-WX-1
14 compounds ends with character info cannot be found in csv
ARD-3461-WX
BD-7216-HJG
ERD-12310-B
ARD-1745-WX
M-1108-XSL
ERD-1237-D-RKA
M-2259-HZ
ARD-2889-WX
M-2272-AA
ARD-2063-WX
ERD-1237-C-RKA
ARD-1138-WX
M-1773-AA
ARD-1735-WX
1 compounds ends with num yet not in -num form
['A947']
11 compounds ends with num > 20
ARV-471-23
PVT-004206
PVT-4204
DI-1589
ARV-471-22
WDR5-3145
ARV-471-21
SNDX-5613
ARV-471-24
EEDI-1251
PVT-3685
*************************************************************

In [19]:
t_dict = {}
from tqdm import tqdm
invalid_kp_names = []
valid_kp_names = []
one_tissue_names = []
only_plasma_names = []
specials = []
for name in tqdm(names):
    # print(name)
    try:
        data = pk(name)
        l = data.get_kp()
        name, df, tissues, _, dose_routes, doses, time_points, species = data.get_info()
        if len(l) == 0:
            # print(name)
            invalid_kp_names.append(name)
            if len(tissues) == 1:
                one_tissue_names.append(name)
                if tissues[0] == 'Plasma':
                  only_plasma_names.append(name)
            else:
                specials.append(name)
            # name, data_df, tissues, targets, dose_routes, doses, time_points, species = data.get_info()
        # for t in time_points:
        #     if t not in t_dict.keys():
        #         t_dict[t] = 1
        #     else: t_dict[t] += 1
        else:
            valid_kp_names.append(name)
    except:
        print(f'cannot handle {name}')
print()

print(f'total {len(names)} compounds, '
      f"can calculate {len(valid_kp_names)} compounds,",
      f"cannot handle {len(invalid_kp_names)} compounds,\n",
      f"* in invalid calculations {len(one_tissue_names)} compounds have one tissue only \n",
      f"* in those one tissue drugs, {len(only_plasma_names)} compounds have plasma only \n",
      f"* still {len(specials)} compounds need to check")



# specials, 5 compounds indeed no kp:
# 'AK-2288', liver == 0
# 'AK-2278', dosage different, cannot calculate kp
# ARD-3007-WX. BLQ
# M-1403-XRQ, BLQ
# SD-3016 BLQ
################################

100%|██████████| 1382/1382 [00:34<00:00, 40.55it/s]


total 1382 compounds, can calculate 680 compounds, cannot handle 702 compounds,
 * in invalid calculations 696 compounds have one tissue only 
 * in those one tissue drugs, 667 compounds have plasma only 
 * still 6 compounds need to check


In [26]:
ind = 23
name = valid_kp_names[ind]
print(name)
pd.DataFrame.from_dict(pk(name).get_kp())

SMD-3040-YL-5


,dose_route,dose,time,animal,tissue,Kp
0,IV,50,24,M2,Tumor,14.470199
1,IV,50,24,M3,Tumor,8.227273
2,IV,50,24,M1,Tumor,12.808642
3,IV,50,48,M2,Tumor,9.808511
4,IV,50,48,M3,Tumor,38.823529
5,IV,50,48,M1,Tumor,10.920000
6,IV,50,3,M2,Tumor,2.813830
7,IV,50,3,M3,Tumor,0.445455
8,IV,50,3,M1,Tumor,0.377193
9,IV,50,96,M2,Tumor,6.343669


In [28]:
# d = 0
in_smile_excel = []
in_smile_excel_after_convertion = []
not_in_excel = []
valid_kp_dict = {}
for i in tqdm(valid_kp_names):
    if i in smiles_df['NAME'].tolist():
        # name_dict[i] = i
        valid_kp_dict[i] = i
        in_smile_excel.append(i)
        pass
    else:
        try:
            if name_dict[i] in smiles_df['NAME'].tolist():
                in_smile_excel_after_convertion.append(i)
                valid_kp_dict[i] = name_dict[i]
        except:
            not_in_excel.append(i)
        # print(i)
        # d += 1
# print(d)
print()
print(f'total {len(valid_kp_names)} compounds having kp, \n'
      f"*{len(in_smile_excel)} compounds in {smi_path},\n",
      f"*{len(in_smile_excel_after_convertion)} compounds in {smi_path}",
       "after convertion (either delete parenthesis, or delete -num),\n")
print(f'thus total {len(valid_kp_dict)} compounds could be proceeded')

print('*' * 80)
print(f"*{len(not_in_excel)} compounds are not in {smi_path}:\n")
for i in not_in_excel:
    print(i)
print('*' * 80)


100%|██████████| 680/680 [00:00<00:00, 25738.43it/s]


total 680 compounds having kp, 
*367 compounds in drug_smile.xlsx,
 *288 compounds in drug_smile.xlsx after convertion (either delete parenthesis, or delete -num),

thus total 655 compounds could be proceeded
********************************************************************************
*25 compounds are not in drug_smile.xlsx:

ARV-471-23
ARD-3461-WX
ERD-1297-RKA-1
PVT-004206
PVT-4204
BD-7216-HJG
DI-1589
ERD-12310-B
ARV-471-22
ERD-12310-B-1
BD-7216-HJG-1
ARV-471-21
ERD-1237-D-RKA
ARD-2889-WX
ARD-2063-WX
ERD-12310-B-2
EEDI-1251-1
SNDX-5613
BD-7216-HJG-2
ERD-1237-RKA-8
ARV-471-24
ERD-1237-C-RKA
BD-7216-HJG-3
PVT-3685
ERD-1233-RKA-10
********************************************************************************


In [38]:
from statistics import mean
certain_time = "6" #@param ["24", "6"]
certain_time = int(certain_time)
certain_time_kp_names = []
kp_list = []
tissue = "Tumor" #@param ["Tumor", "Spleen", "Liver"]
route = "IV" #@param ["PO", "IV", "IP"]

new_df = pd.DataFrame()
for i, j in tqdm(valid_kp_dict.items(),
                 desc='valid kp compounds also have SMILES and property info in csv'):
    # j = name_dict[i]
    # df2 = df.loc[:, df.columns != "Duration"]
    features = smiles_df[smiles_df['NAME'] == j].iloc[:, 1:]
    # features = features.loc[:, features.columns != 'SMILES']
    if features.shape[0] > 1:  # duplicates in drug_smile file
        # print(features)
        features = features.iloc[0]
        try:
            features = features.values.reshape(1, int(features.size))
            features = pd.DataFrame(features, columns = smiles_df.columns[3:])
            # features['NAME'] = i
            # features['']
            assert features.shape[0] == 1
        except:
            print('now features,size: ', features.size)
            print('now features.shape: ', features.shape)
            pass

    data = pk(i)
    name, df, tissues, target, dose_routes, doses, \
    time_points, species = data.get_info()
    all_kps = data.get_kp()
    df = pd.DataFrame.from_dict(all_kps)


    tumor_df = df.loc[(df['tissue'] == tissue) & \
                      (df['time'] == certain_time) & \
                      (df['dose_route'] == route)]
    if tumor_df.shape[0] != 0:
        # print(tumor_df)
        certain_time_kp_names.append(i)
        new_df = new_df.append([features]*1,ignore_index=True)
        kp_list.append(mean(tumor_df['Kp'].tolist()))
        try:
            assert new_df.shape[0] == len(kp_list)
            assert tumor_df.shape[0] == len(tumor_df['Kp'].tolist())
        except:
            print('cannot handle: ', i, j)
            break
print()
print(len(certain_time_kp_names),
      f'compounds have {certain_time}h tumor Kp '
      f'and {route} method of delivery')
new_df['Kp_average'] = pd.DataFrame(kp_list)
pd.set_option('display.max_columns', None)
new_df.to_csv(f'Kp_data/{certain_time}_Kp_{tissue}_{route}.csv', index=False)
new_df

# print('Kp tumor at 24 h: ')
# print(new_df)

valid kp compounds also have SMILES and property info in csv: 100%|██████████| 655/655 [00:30<00:00, 21.34it/s]



55 compounds have 6h tumor Kp and IV method of delivery


,NAME,SMILES,MW,ALOGP,HBA,HBD,PSA,ROTB,AROM,ALERTS,QED,Kp_average
0,SD-91,O=C(CCC1N2CC3=C(C2=O)C=CC=C3C#CCCCCCC(N4C[C@@H...,1136.169,3.59540,13,8,327.88,19,5,4,0.025405,0.140289
1,AK-2292,CN(C(CCN(C1=CC=C(C2=NC=CS2)C=C1)C([C@@H]3CCCN3...,1070.147,6.86190,11,4,226.93,15,5,4,0.036657,3.974948
2,BD-7163-HJG,O=C1N(C2CCC(NC2=O)=O)C(C3=C(N4CCC(N5C=C(C#CC6=...,726.819,4.09722,9,1,144.55,5,5,2,0.209233,0.856876
3,AK-2292,CN(C(CCN(C1=CC=C(C2=NC=CS2)C=C1)C([C@@H]3CCCN3...,1070.147,6.86190,11,4,226.93,15,5,4,0.036657,4.177881
4,BM-1250,O=S(NC(C=C1)=CC=C1N2CCN(C3=CC(C4=C(C5=CC=C(Cl)...,1267.846,10.72412,16,4,224.96,22,6,4,0.021459,1.627341
5,SD-373,O=C1[C@@H](NC(C2=CC(C=C(C(F)(F)P(O)(O)=O)C=C3)...,1160.248,3.56870,13,7,298.26,16,3,3,0.061430,0.137862
6,SD-382,O=C1[C@@H](NC(C2=CC(C=C(C(P(O)(O)=O)=O)C=C3)=C...,1153.221,4.32880,13,7,312.09,19,5,4,0.026254,0.136098
7,SMD-3236-YL,ClC1=CC=CC(N23)=C1C(N=C3C4(C5=C2C=C(C6CCN(C[C@...,1095.856,9.51822,11,2,163.84,12,6,0,0.120024,1.074676
8,SMD-3050-YL,CC(C)(C)[C@@H](C(N1[C@H](C(N[C@@H](C)C2=CC=C(C...,973.302,6.61832,12,5,179.39,11,4,0,0.108751,4.168982
9,SD-36,O=C(N1)CCC(N2CC(C(C#CCCCCCC(N3CC(NC(C4=CC(C=C(...,1158.166,4.50450,12,8,310.81,19,5,4,0.024536,0.301055
